In [ ]:
import pandas as pd
import numpy as np

# Reload centrality scores

In [ ]:
def load_score_map(score_folder,day, measure):
    """The centrality maps were pre-sorted in decreasing order!!!"""
    scores = pd.read_csv(score_folder + '/%s_scores_%i.txt_s' % (measure,day), sep=" ", names=["id","score_%i" % day])
    scores = scores.set_index("id")
    return scores

In [ ]:
def load_score_maps(score_folder, days, measure):
    """Load daily score maps. Then join them by ID."""
    daily_scores = load_score_map(score_folder,days[0],measure)
    for i in range(1,len(days)):
        print i, len(daily_scores)
        current_scores = load_score_map(score_folder,days[i],measure)
        daily_scores = daily_scores.join(current_scores, how='outer')
    return daily_scores.fillna(0.0)

In [ ]:
dataset_id = "olympics"

#measure_prefix = "in_degree"
measure_prefix = "pagerank"
input_folder = "/mnt/idms/fberes/NETWORK/DATA/temporal_centralities/centrality_output_for_datasets_normalized/%s/centrality_scores/" % dataset_id

#measure_prefix = "popularity_model"
#input_folder = "/mnt/idms/fberes/NETWORK/andreas_article/nelly_model_scores/%s/centrality_scores/" % dataset_id

days = range(21)

print measure_prefix
print days
print input_folder

In [ ]:
scores = load_score_maps(input_folder,days,measure=measure_prefix)

In [ ]:
scores.columns

In [ ]:
scores.head()

# Export centrality scores (learning to rank)

### Parameters to set:

   * Number of features (4-8) - this determines the available number of queries
   * unseen true/false : the fatures cannot be all zeros = unseenTrue
   * top_k : the first k biggest label is included in the records
   * the output have to be splitted for train and test (later by hand)

In [ ]:
def generate_learning_to_rank_input(score_df, num_of_features, k, unseen=True):
    num_cols = score_df.columns
    output_records = np.array([])
    for i in range(num_of_features, len(num_cols)):
        partial_score_mx = score_df.sort('score_%i' % i, ascending=0).head(k).as_matrix()
        features_and_labels = partial_score_mx[:,i-num_of_features:i+1]
        if not unseen: # filter records where all features are zero
            filtered = []
            for j in xrange(len(features_and_labels)):
                if not np.array_equal(features_and_labels[j,:-1],[0]*num_of_features):
                    filtered.append(list(features_and_labels[j,:]))
            features_and_labels = np.array(filtered)
        partial_output = np.zeros((len(features_and_labels),num_of_features+2))
        partial_output[:,0] = i-num_of_features+1
        partial_output[:,1:] = features_and_labels
        if i == num_of_features:
            output_records = partial_output
        else:
            output_records = np.concatenate((output_records,partial_output),axis=0)
        #print partial_output.shape, output_records.shape
    return output_records 

In [ ]:
if measure_prefix == "popularity_model":
    l2r_output_folder = '/mnt/idms/fberes/NETWORK/andreas_article/learning_to_rank_inputs/popularity_model/'
else:
    l2r_output_folder = '/mnt/idms/fberes/NETWORK/andreas_article/learning_to_rank_inputs/original_datasets/'
#os.makedirs(l2r_output_folder)
print l2r_output_folder

In [ ]:
for feat in [4]:
    for k in [100,500]:
        for unseen in [True,False]:
            all_output = generate_learning_to_rank_input(scores, feat, k, unseen=unseen)
            output_df = pd.DataFrame(all_output)
            output_df[0] = output_df[0].astype('int')
            #print len(output_df)
            output_df.to_csv(l2r_output_folder + "%s_%s_k%i_f%i_unseen%s.csv" % (dataset_id,measure_prefix,k,feat,unseen), sep=";",index=False,header=False,float_format='%.10f')